In [9]:
import numpy as np
import scipy as sci

In [6]:
try:
    import pyopencl as cl
    import pyopencl.array as cl_array
    from pyopencl.clrandom import rand as clrand
except ImportError:
    Warning("PyOpenCl not installed")
import os
from mako.template import Template

In [7]:
class AWGN_channel:
    """ Class implements an additive white Gaussian noise channel

    The added noise can either be real or complex depending on the arguments of the constructor.
    The default value is real.

    Attributes:
        sigma_n2: a double setting noise variance
        complex: a boolean value indicating if noise is complex or not
    """
    def __init__(self, sigma_n2_, complex =False):
        """Inits the AWGN_channel class
        Args:
            sigma_n2_: noise variance specified by user
            complex: default is false, indicating if noise is complex
        """
        self.sigma_n2 = sigma_n2_
        self.complex = complex

    def transmission(self, input):
        """Performs the transmission of an input stream over an AWGN channel
        Args:
            input: sequence of symbols as numpy array or scalar
        Returns:
            output: summation of noise and input
        """
        if self.complex:
            noise = np.sqrt(self.sigma_n2/2) * np.random.randn(input.shape[0], input.shape[1]) + \
                    1j * np.sqrt(self.sigma_n2/2) * np.random.randn(input.shape[0], input.shape[1])

        else:
            noise = np.sqrt(self.sigma_n2) * np.random.randn(input.shape[0],input.shape[1])

        output = input + noise

        return output

In [8]:
class AWGN_Channel_Quantizer:
    """Implementation of an information optimum quantizer unit assuming BPSK transmission.

    The quantizer is generated using the symmetric, sequential information bottleneck algorithm.
    This class supports OpenCL for faster quantization and even direct quantization and sample generation on the GPU
    (cf. quantize direct).
    Although it is theoretical correct to quantize directly, it is preferable to create a more realistic
    communication chain including an encoder and modulator in your system instead of using this direct quantization approach.

    Attributes:
        sigma_n2: noise variance corresponding to the desired design-Eb/N0 of the decoder
        AD_max_abs: limits of the quantizer
        cardinality_Y: number of steps used for the fine quantization of the input distribution of the quantizer
        cardinality_T: cardinality of the compression variable representing the quantizer output

        limits: borders of the quantizer regions
        y_vec: fine quantization of the input domain
        delta: spacing between two values in the quantized input domain (cf. y_vec)

        x_vec: position of the means of the involved Gaussians

    """
    def __init__(self, sigma_n2_, AD_max_abs_, cardinality_T_, cardinality_Y_, dont_calc = False):
        """Inits the quantizer class."""
        self.nror = 5
        self.limits = np.zeros(cardinality_T_)

        self.sigma_n2 = sigma_n2_
        self.cardinality_T = cardinality_T_
        self.cardinality_Y = cardinality_Y_
        self.AD_max_abs = AD_max_abs_

        self.y_vec = np.linspace(-self.AD_max_abs, +self.AD_max_abs, self.cardinality_Y)
        self.x_vec = np.array([-1, 1])
        self.delta = self.y_vec[1] - self.y_vec[0]
        if not dont_calc:
            self.calc_quanti()

    def calc_quanti(self):
        """Determines the information optimum quantizer for the given input distribution"""

        # calculate p_xy based on sigma_n2 and AD_max_abs;
        # init as normal with mean + 1
        p_y_given_x_equals_zero = norm.pdf(self.y_vec, loc=1, scale=np.sqrt(self.sigma_n2)) * self.delta

        # truncate t account for distortion introduced by the quantizer limits then
        p_y_given_x_equals_zero[-1] += self.gaussian_over_prob(self.AD_max_abs, 1)
        p_y_given_x_equals_zero[0] += self.gaussian_under_prob(-self.AD_max_abs, 1)

        # flip distribution, which realizes mean -1 or a transmitted bit = 1
        p_y_given_x_equals_one = p_y_given_x_equals_zero[::-1]

        self.p_xy = 0.5 * np.hstack((p_y_given_x_equals_zero[:,np.newaxis], p_y_given_x_equals_one[:,np.newaxis]))

        self.p_xy = self.p_xy / self.p_xy.sum() #normalize for munerical stability

        # run the symmetric sequential Information Bottleneck algorithm
        IB_class = symmetric_sIB(self.p_xy, self.cardinality_T, self.nror)
        IB_class.run_IB_algo()

        # store the results
        [self.p_t_given_y, self.p_x_given_t, self.p_t] = IB_class.get_results()

        # calculate
        # p(t | X = 0)=p(X=0 | t)
        # p(t) / p(X=0)
        self.p_x_given_t = self.p_x_given_t / self.p_x_given_t.sum(1)[:,np.newaxis]
        self.p_x_and_t = self.p_x_given_t * self.p_t[:,np.newaxis]
        p_t_given_x_equals_zero = self.p_x_and_t[:, 0] / 0.5

        self.cdf_t_given_x_equals_zero = np.append([0], np.cumsum(p_t_given_x_equals_zero))

        self.output_LLRs = np.log(self.p_x_and_t[:, 0] / self.p_x_and_t[:, 1])
        self.calc_limits()

    @classmethod
    def from_generated(cls, cdf_t_given_x_equals_zero_):
        cdf_t_given_x_equals_zero = cdf_t_given_x_equals_zero_
        return cls(cdf_t_given_x_equals_zero,)

    def gaussian_over_prob(self, x, mu):
        """Compensates the ignored probability mass caused by fixing the region to +- AD_abs_max."""

        prob = norm.sf((x-mu+self.delta/2)/np.sqrt(self.sigma_n2))
        return prob

    def gaussian_under_prob(self, x, mu):
        """Compensates the ignored probability mass caused by fixing the region to +- AD_abs_max."""

        prob = 1-self.gaussian_over_prob(x-self.delta,mu)
        return prob

    def calc_limits(self):
        """Calculates the limits of the quantizer borders"""

        for i in range(self.cardinality_T):
            cur_vec = (self.p_t_given_y[:, i] == 1).nonzero()
            self.limits[i] = self.y_vec[cur_vec[0].min()]

        self.limits[int(self.cardinality_T/2)] = 0
        #self.limits[-1]=self.AD_max_abs

    def quantize_direct(self, input_bits):
        """Direct quantization without the need of a channel in between since the inversion method is used.
        The clusters are directly sampled.
        """
        # create uniform samples
        rand_u = np.random.rand(input_bits.shape[0],input_bits.shape[1])

        # create samples ~ p(t | X = 0) using inversion method
        if input_bits.shape[1] > 1:
            output_integers = ((np.repeat(rand_u[:,:,np.newaxis], self.cardinality_T+1, axis=2)-self.cdf_t_given_x_equals_zero) > 0).sum(2)-1
            output_integers[input_bits.astype(bool)] = self.cardinality_T - 1 - output_integers[input_bits.astype(bool)]
        else:
            output_integers = ((rand_u - self.cdf_t_given_x_equals_zero) > 0).sum(1) - 1
            # "mirror" a sample, when the input bit is 1, otherwise do nothing.
            output_integers[input_bits.astype(bool)[:, 0]] = self.cardinality_T - 1 - output_integers[
            input_bits.astype(bool)[:, 0]]

        return output_integers

    def quantize_on_host(self,x):
        """Quantizes the received samples on the local machine"""
        if x.shape[1] > 1:
            cluster = ((np.repeat(x[:,:,np.newaxis], self.cardinality_T, axis=2)-self.limits) > 0).sum(2)-1
            cluster[cluster == -1] = 0
        else:
            cluster = np.sum((x - self.limits) > 0, 1) -1
            cluster[cluster==-1] = 0

        return cluster

    def init_OpenCL_quanti(self, N_var,msg_at_time,return_buffer_only=False):
        """Inits the OpenCL context and transfers all static data to the device"""

        self.context = cl.create_some_context()

        print(self.context.get_info(cl.context_info.DEVICES))
        path = os.path.split(os.path.abspath(__file__))
        kernelsource = open(os.path.join(path[0], 'kernels_quanti_template.cl')).read()

        tpl = Template(kernelsource)
        rendered_tp = tpl.render(Nvar=N_var)

        self.program = cl.Program(self.context, str(rendered_tp)).build()

        self.return_buffer_only = return_buffer_only

        # Set up OpenCL
        self.queue = cl.CommandQueue(self.context)
        self.quantize = self.program.quantize
        self.quantize.set_scalar_arg_dtypes([np.int32, None, None, None])
        self.quantize_LLR = self.program.quantize_LLR
        self.quantize_LLR.set_scalar_arg_dtypes([np.int32, None, None, None,None])
        self.limit_buff = cl_array.to_device(self.queue, self.cdf_t_given_x_equals_zero.astype(np.float64))
        self.cluster_buff = cl_array.empty(self.queue, (N_var, msg_at_time), dtype=np.int32)
        self.LLR_buff = cl_array.empty(self.queue, (N_var, msg_at_time), dtype=np.float64)
        self.LLR_values_buff = cl_array.to_device(self.queue, self.output_LLRs.astype(np.float64))

    def quantize_OpenCL(self, x):
        """Quantizes the received distorted samples on the graphic card"""

        # Create OpenCL buffers

        x_buff = cl_array.to_device(self.queue,x.astype(np.float64) )
        limit_buff = cl_array.to_device(self.queue, self.limits.astype(np.float64))
        cluster_buff = cl_array.empty_like(x_buff.astype(np.int32))

        self.quantize(self.queue, x.shape, None, self.cardinality_T, x_buff.data, limit_buff.data, cluster_buff.data)
        self.queue.finish()

        if self.return_buffer_only:
            return cluster_buff
        else:
            clusters = cluster_buff.get()
            return clusters

    def quantize_direct_OpenCL(self,N_var,msg_at_time):
        """Direct quantization without the need of a channel in between since the inversion method is used.
        The clusters are directly sampled. In this scenario the all-zeros codeword is considered such that no data
        needs to be transferred to the graphic card.

        """

        #rand_u_buff = clrand(self.queue, (N_var,msg_at_time), dtype=np.float64, a=0, b=1)

        rand_u = np.random.rand(N_var,msg_at_time)

        # Create OpenCL buffers

        rand_u_buff = cl_array.to_device(self.queue,rand_u.astype(np.float64) )



        self.quantize(self.queue, (N_var,msg_at_time), None, self.cardinality_T+1, rand_u_buff.data,
                      self.limit_buff.data, self.cluster_buff.data)


        self.queue.finish()

        if self.return_buffer_only:
            return self.cluster_buff
        else:
            clusters = self.cluster_buff.get()
            return clusters

    def quantize_direct_OpenCL_LLR(self,N_var,msg_at_time):
        """ Returns the LLRs of the sampled cluster indices. These indices correspond to the quantized outputs which
        are found directly on the graphic card using the inversion method. """

        rand_u = np.random.rand(N_var,msg_at_time)

        # Create OpenCL buffers
        rand_u_buff = cl_array.to_device(self.queue,rand_u.astype(np.float64) )

        self.quantize_LLR(self.queue, (N_var,msg_at_time), None, self.cardinality_T+1, rand_u_buff.data,
                      self.limit_buff.data, self.LLR_values_buff.data, self.LLR_buff.data)

        self.queue.finish()

        if self.return_buffer_only:
            return self.LLR_buff
        else:
            LLRs = self.LLR_buff.get()
            return LLRs

In [10]:
class LDPC_BPSK_Transmitter:

    def __init__(self, filename_H_,  msg_at_time=1):
        self.filename_H = filename_H_
        self.H_sparse = self.load_check_mat(self.filename_H)

        self.encoder = LDPCEncoder(self.filename_H)

        # analyze the H matrix and set all decoder variables
        self.set_code_parameters()

        self.data_len = (self.R_c * self.codeword_len).astype(int)

        self.last_transmitted_bits = []
        self.msg_at_time = msg_at_time

    def set_code_parameters(self):
        self.degree_checknode_nr = ((self.H_sparse).sum(1)).astype(np.int).A[:, 0]  # which check node has which degree?
        self.degree_varnode_nr = ((self.H_sparse).sum(0)).astype(np.int).A[0,
                                 :]  # which variable node has which degree?

        self.N_v = self.H_sparse.shape[1]  # How many variable nodes are present?
        self.N_c = self.H_sparse.shape[0]  # How many checknodes are present?

        self.d_c_max = self.degree_checknode_nr.max()
        self.d_v_max = self.degree_varnode_nr.max()

        self.codeword_len = self.H_sparse.shape[1]
        row_sum = self.H_sparse.sum(0).A[0, :]
        col_sum = self.H_sparse.sum(1).A[:, 0]
        d_v_dist_val = np.unique(row_sum)
        d_v_dist = np.zeros(int(d_v_dist_val.max()))

        for d_v in np.sort(d_v_dist_val).astype(np.int):
            d_v_dist[d_v - 1] = (row_sum == d_v).sum()
        d_v_dist = d_v_dist / d_v_dist.sum()

        d_c_dist_val = np.unique(col_sum)
        d_c_dist = np.zeros(int(d_c_dist_val.max()))

        for d_c in np.sort(d_c_dist_val).astype(np.int):
            d_c_dist[d_c - 1] = (col_sum == d_c).sum()

        d_c_dist = d_c_dist / d_c_dist.sum()
        nom = np.dot(d_v_dist, np.arange(d_v_dist_val.max()) + 1)
        den = np.dot(d_c_dist, np.arange(d_c_dist_val.max()) + 1)

        self.R_c = 1 - nom / den

    def alistToNumpy(self, lines):
        """Converts a parity-check matrix in AList format to a 0/1 numpy array. The argument is a
       list-of-lists corresponding to the lines of the AList format, already parsed to integers
        if read from a text file.
        The AList format is introduced on http://www.inference.phy.cam.ac.uk/mackay/codes/alist.html.
        This method supports a "reduced" AList format where lines 3 and 4 (containing column and row
        weights, respectively) and the row-based information (last part of the Alist file) are omitted.
        Example:
             >>> alistToNumpy([[3,2], [2, 2], [1,1,2], [2,2], [1], [2], [1,2], [1,2,3,4]])
            array([[1, 0, 1],
                  [0, 1, 1]])
        """

        nCols, nRows = lines[0]
        if len(lines[2]) == nCols and len(lines[3]) == nRows:
            startIndex = 4
        else:
            startIndex = 2
        matrix = np.zeros((nRows, nCols), dtype=np.int)
        for col, nonzeros in enumerate(lines[startIndex:startIndex + nCols]):
            for rowIndex in nonzeros:
                if rowIndex != 0:
                    matrix[rowIndex - 1, col] = 1

        return matrix

    def load_sparse_csr(self, filename):
        loader = np.load(filename)
        return sci.sparse.csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                              shape=loader['shape'])

    def load_check_mat(self, filename):
        if filename.endswith('.npy') or filename.endswith('.npz'):
            if filename.endswith('.npy'):
                H = np.load(filename)
                H_sparse = sci.sparse.csr_matrix(H)
            else:
                H_sparse = self.load_sparse_csr(filename)
        else:
            arrays = [np.array(list(map(int, line.split()))) for line in open(filename)]
            H = self.alistToNumpy(arrays)
            H_sparse = sci.sparse.csr_matrix(H)
        return H_sparse

    def transmit(self):

        uncoded_msgs = np.random.randint(0,2, (self.data_len, self.msg_at_time))

        #uncoded_msgs = np.zeros( (self.data_len, self.msg_at_time) )
        encoded_msgs = np.zeros((self.codeword_len, self.msg_at_time))


        for i in range(self.msg_at_time):

            encoded_msgs[:, i]=self.encoder.encode_c(uncoded_msgs[:, i])

        self.last_transmitted_bits = uncoded_msgs

        data = self.BPSK_mapping(encoded_msgs)

        return data

    def BPSK_mapping(self, X):

        data = np.ones((self.codeword_len, self.msg_at_time))
        data[X == 1] = -1

        return data

In [11]:
class AWGN_Discrete_Density_Evolution_class:
    """ Generates a discrete LDPC decoder for a AWGN channel and a regular LDPC code for a certain design-Eb/N0.

    The assumed modulation is BPSK which is considered in the quantizer design.
    Attributes:
        sigma_n2: noise variance corresponding to the desired design-Eb/N0 of the decoder
        AD_max_abs: limits of the quantizer
        cardinality_Y_channel: number of steps used for the fine quantization of the input distribution of the quantizer
        cardinality_T_channel: cardinality of the compression variable representing the quantizer output

        cardinality_T_decoder_ops: cardinality of the compression variables inside the decoder

        d_c: check node degree
        d_v: variable node degree

        imax: maximum number of iterations
        nror: number of runs of the Information Bottleneck algorithm

        Trellis_checknodevector_a:  vectorized version of the trellis which holds the resulting outputs for a certain
                                    input and iteration at a check node
        Trellis_varnodevector_a:  vectorized version of the trellis which holds the resulting outputs for a certain
                                    input and iteration at a variable node
    """
    def __init__(self, sigma_n2_, AD_max_abs_,cardinality_Y_channel_, cardinality_T_channel_,
                 cardinality_T_decoder_ops_,d_v_, d_c_, i_max_, nror_):
        """Inits the AWGN_Discrete_Density_Evolution_class with the following arguments

        Args:
            sigma_n2_: noise variance corresponding to the desired design-Eb/N0 of the decoder
            AD_max_abs_: limits of the quantizer
            cardinality_Y_channel_: number of steps used for the fine quantization of the input distribution of the quantizer
            cardinality_T_channel_: cardinality of the compression variable representing the quantizer output

            cardinality_T_decoder_ops_: cardinality of the compression variables inside the decoder

            d_c_: check node degree
            d_v_: variable node degree

            i_max_: maximum number of iterations
            nror_: number of runs of the Information Bottleneck algorithm
        """
        # copy input arguments to class attributes
        self.sigma_n2 = sigma_n2_
        self.AD_max_abs = AD_max_abs_

        self.cardinality_Y_channel = cardinality_Y_channel_
        self.cardinality_T_channel = cardinality_T_channel_
        self.cardinality_T_decoder_ops = cardinality_T_decoder_ops_

        self.d_v = d_v_
        self.d_c = d_c_

        R_c = 1 - self.d_v / self.d_c
        if R_c > 0:
            self.EbN0 = -10 * np.log10(self.sigma_n2 * 2 * R_c)

        self.imax = i_max_
        self.nror = nror_

        self.build_quantizer()


        self.Trellis_checknodevector_a = 0
        self.Trellis_varnodevector_a = 0

    def set_code_parameters(self):
        """Analysis of the given parity check matrix.
        Determines node-degree distribution, edge-degree distribution and code rate
        """
        self.degree_checknode_nr = ((self.H_sparse).sum(1)).astype(np.int).A[:, 0]  # which check node has which degree?
        self.degree_varnode_nr = ((self.H_sparse).sum(0)).astype(np.int).A[0,
                                 :]  # which variable node has which degree?

        self.N_v = self.H_sparse.shape[1]  # How many variable nodes are present?
        self.N_c = self.H_sparse.shape[0]  # How many checknodes are present?

        self.d_c_max = self.degree_checknode_nr.max()
        self.d_v_max = self.degree_varnode_nr.max()

        self.codeword_len = self.H_sparse.shape[1]
        row_sum = self.H_sparse.sum(0).A[0, :]
        col_sum = self.H_sparse.sum(1).A[:, 0]
        d_v_dist_val = np.unique(row_sum)
        d_v_dist = np.zeros(int(d_v_dist_val.max()))

        for d_v in np.sort(d_v_dist_val).astype(np.int):
            d_v_dist[d_v - 1] = (row_sum == d_v).sum()

        d_v_dist = d_v_dist / d_v_dist.sum()

        d_c_dist_val = np.unique(col_sum)
        d_c_dist = np.zeros(int(d_c_dist_val.max()))

        for d_c in np.sort(d_c_dist_val).astype(np.int):
            d_c_dist[d_c - 1] = (col_sum == d_c).sum()

        d_c_dist = d_c_dist / d_c_dist.sum()

        nom = np.dot(d_v_dist, np.arange(d_v_dist_val.max()) + 1)
        den = np.dot(d_c_dist, np.arange(d_c_dist_val.max()) + 1)

        self.lambda_vec = convert_node_to_edge_degree(d_v_dist)
        self.rho_vec = convert_node_to_edge_degree(d_c_dist)

        self.R_c = 1 - nom / den

    def alistToNumpy(self, lines):
        """Converts a parity-check matrix in AList format to a 0/1 numpy array. The argument is a
        list-of-lists corresponding to the lines of the AList format, already parsed to integers
        if read from a text file.
        The AList format is introduced on http://www.inference.phy.cam.ac.uk/mackay/codes/alist.html.
        This method supports a "reduced" AList format where lines 3 and 4 (containing column and row
        weights, respectively) and the row-based information (last part of the Alist file) are omitted.
        Example:
             >>> alistToNumpy([[3,2], [2, 2], [1,1,2], [2,2], [1], [2], [1,2], [1,2,3,4]])
            array([[1, 0, 1],
                  [0, 1, 1]])
        """

        nCols, nRows = lines[0]
        if len(lines[2]) == nCols and len(lines[3]) == nRows:
            startIndex = 4
        else:
            startIndex = 2
        matrix = np.zeros((nRows, nCols), dtype=np.int)
        for col, nonzeros in enumerate(lines[startIndex:startIndex + nCols]):
            for rowIndex in nonzeros:
                if rowIndex != 0:
                    matrix[rowIndex - 1, col] = 1

        return matrix

    def load_sparse_csr(self, filename):
        """Performs loading of a sparse parity check matrix which is stored in a *.npy file."""
        loader = np.load(filename)
        return sci.sparse.csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                                     shape=loader['shape'])

    def load_check_mat(self, filename):
        """Performs loading of a predefined parity check matrix."""
        if filename.endswith('.npy') or filename.endswith('.npz'):
            if filename.endswith('.npy'):
                H = np.load(filename)
                H_sparse = sci.sparse.csr_matrix(H)
            else:
                H_sparse = self.load_sparse_csr(filename)
        else:
            arrays = [np.array(list(map(int, line.split()))) for line in open(filename)]
            H = self.alistToNumpy(arrays)
            H_sparse = sci.sparse.csr_matrix(H)
        return H_sparse

    def build_quantizer(self):
        """Generates instance of a quantizer for BPSK and an AWGN channel for the given characteristics."""
        quanti = AWGN_Channel_Quantizer_BPSK(self.sigma_n2,self.AD_max_abs,self.cardinality_T_channel,self.cardinality_Y_channel)
        self.p_x_and_t_input = quanti.p_x_and_t

    def run_discrete_density_evolution(self):
        """Performs the discrete density evolution using the input distributions obtained from the quantizer.
        The resulting trellis diagram is stored in a vector that can be used for the real decoder later.
        """
        DDE_inst = discrete_DE(self.p_x_and_t_input, self.cardinality_T_decoder_ops,
                               self.d_v, self.d_c, self.imax, self.nror)

        DDE_inst.run_discrete_Density_Evolution()

        self.Trellis_checknodevector_a = DDE_inst.Trellis_checknodevector_a
        self.Trellis_varnodevector_a = DDE_inst.Trellis_varnodevector_a

        self.DDE_inst_data = DDE_inst.__dict__

    def save_config(self,text=''):
        """Saves the instance."""
        #timestr = time.strftime("%Y%m%d-%H%M%S")
        timestr =''

        output = open('decoder_config_EbN0_gen_' + str(self.EbN0) + '_' + str(
            self.cardinality_T_decoder_ops) + timestr + text + '.pkl', 'wb')

        # Pickle dictionary using protocol -1.
        pickle.dump(self.__dict__, output, protocol=-1)

class AWGN_Discrete_Density_Evolution_class_irregular(AWGN_Discrete_Density_Evolution_class):
    """Inherited from base class AWGN_Discrete_Density_Evolution_class.

    Generalization for irregular codes. Thus a new discrete density evolution schemes is used.

    Attributes:
        filename_H: filename of the parity check matrix of the considered code
        H_sparse: corresponding parity check matrix for the considered code

        matching_vector_varnode: holds the deterministic mapping found when performing message alginment for a
                                 variable noce
        matching_vector_checknode: holds the deterministic mapping found when performing message alginment for a
                                   check node

        match: boolean indicating if alignment should be used or not
    """
    def __init__(self, sigma_n2_, AD_max_abs_,cardinality_Y_channel_, cardinality_T_channel_,
                 cardinality_T_decoder_ops_,filename_H_, i_max_, nror_,match=True):
        """Inits AWGN_Discrete_Density_Evolution_class_irregular class.

        Args:
            filename_H_: filename of parity check matrix
            match: boolean indicating if alignment should be used or not
        """
        self.filename_H = filename_H_
        self.H_sparse = self.load_check_mat(self.filename_H)
        self.set_code_parameters()
        AWGN_Discrete_Density_Evolution_class.__init__(self, sigma_n2_,
                                                       AD_max_abs_, cardinality_Y_channel_, cardinality_T_channel_,
                                                       cardinality_T_decoder_ops_, self.d_v_max, self.d_c_max ,
                                                       i_max_, nror_)

        self.EbN0 = -10 * np.log10(self.sigma_n2 * 2 * self.R_c)
        self.match = match

    def run_discrete_density_evolution(self):
        """Runs discrete density evolution for irregular codes

        Returns also two matching vectors describing the deterministic transformation obtained by message alingment
        """
        DDE_inst = discrete_DE_irregular(self.p_x_and_t_input, self.cardinality_T_decoder_ops,
                               self.lambda_vec, self.rho_vec, self.imax, self.nror, match=self.match)

        DDE_inst.run_discrete_Density_Evolution()

        self.Trellis_checknodevector_a = DDE_inst.Trellis_checknodevector_a
        self.Trellis_varnodevector_a = DDE_inst.Trellis_varnodevector_a
        self.matching_vector_checknode = DDE_inst.matching_vector_checknode
        self.matching_vector_varnode = DDE_inst.matching_vector_varnode


        self.DDE_inst_data = DDE_inst.__dict__

class AWGN_Discrete_Density_Evolution_class_irregular_QAM(AWGN_Discrete_Density_Evolution_class_irregular):
    """Inherited from base class AWGN_Discrete_Density_Evolution_class_irregular.

    Adapted version for an irregular LDPC code, an AWGN channel and QAM modulation.
    Thus, the quantizer is replaced.
    """

    def __init__(self, sigma_n2_,
                 AD_max_abs_,
                 cardinality_Y_channel_,
                 cardinality_T_channel_,
                 cardinality_T_decoder_ops_,
                 filename_H_,
                 encoding_table,
                 sqrt_M,
                 i_max_,
                 nror_,
                 match=True):
        """Inits AWGN_Discrete_Density_Evolution_class_irregular_QAM class"""

        self.encoding_table = encoding_table
        self.sqrt_M = sqrt_M
        self.num_bits = int(np.log2(sqrt_M) * 2)

        AWGN_Discrete_Density_Evolution_class_irregular.__init__(self,sigma_n2_,
                                                                 AD_max_abs_,
                                                                 cardinality_Y_channel_,
                                                                 cardinality_T_channel_,
                                                                 cardinality_T_decoder_ops_,
                                                                 filename_H_,
                                                                 i_max_,
                                                                 nror_,
                                                                 match)



        self.EbN0 = -10 * np.log10(self.sigma_n2 * self.R_c * self.num_bits)

    def build_quantizer(self):
        """Generates a quantizer of the AWGN channel output where the used modulation scheme is QAM."""

        quanti = AWGN_Channel_Quantizer_QAM(self.sigma_n2,
                                                 self.AD_max_abs,
                                                 self.cardinality_T_channel,
                                                 self.cardinality_Y_channel,
                                                 self.encoding_table,
                                                 sqrt_M=self.sqrt_M )
        self.p_x_and_t_input = quanti.p_b_and_u_matched

class AWGN_Discrete_Density_Evolution_class_QAM(AWGN_Discrete_Density_Evolution_class):
    """Inherited from base class AWGN_Discrete_Density_Evolution_class.

        Adapted version for an regular LDPC code, an AWGN channel and QAM modulation.
        Thus, the quantizer is replaced.
    """

    def __init__(self, sigma_n2_, AD_max_abs_,cardinality_Y_channel_, cardinality_T_channel_,
                 cardinality_T_decoder_ops_,filename_H_,
                 encoding_table,sqrt_M,i_max_, nror_,match=True):

        self.match = match
        self.filename_H = filename_H_
        self.H_sparse = self.load_check_mat(self.filename_H)
        self.set_code_parameters()
        self.encoding_table = encoding_table
        self.sqrt_M = sqrt_M
        self.num_bits = int(np.log2(sqrt_M) * 2)

        AWGN_Discrete_Density_Evolution_class.__init__(self, sigma_n2_,
                                                       AD_max_abs_, cardinality_Y_channel_, cardinality_T_channel_,
                                                       cardinality_T_decoder_ops_, self.d_v_max, self.d_c_max,
                                                       i_max_, nror_)



        self.EbN0 = -10 * np.log10(self.sigma_n2 * self.R_c * self.num_bits)
        pass


    def build_quantizer(self):
        """Generates a quantizer of the AWGN channel output where the used modulation scheme is QAM."""

        quanti = AWGN_Channel_Quantizer_QAM(self.sigma_n2,
                                                 self.AD_max_abs,
                                                 self.cardinality_T_channel,
                                                 self.cardinality_Y_channel,
                                                 self.encoding_table,
                                                 sqrt_M=self.sqrt_M )
        if self.match:
            self.p_x_and_t_input = quanti.p_b_and_u_matched
        else:
            self.p_x_and_t_input = quanti.p_b_and_u_matched_no_match